In [1]:
#This is FEW-SHOT (answer-only) baseline experiment

In [ ]:
import numpy as np
import pandas as pd
import json
import os
import openai


openai.api_key = '' #deleted for upload


#reading in data
df = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/trainwindowsimproved.csv")
df.head()

In [ ]:
df.head()

In [24]:
#Template Construction
def get_template(utterance, examples_df):
    
    metaPrompt = f"""Our definition of vulnerability refers to customers who, due to their personal circumstances, are especially susceptible to harm. All customers are at risk of becoming vulnerable and this risk is increased by characteristics of vulnerability related to 4 key drivers.
    Health - health conditions or illnesses that affect ability to carry out day-to-day tasks.

    Life events - life events such as bereavement, job loss or relationship breakdown.

    Resilience - low ability to withstand financial or emotional shocks.

    Capability - low knowledge of financial matters or low confidence in managing money (financial capability). Low capability in other relevant areas such as literacy, or digital skills
    
    Below is {len(examples_df)} examples of conversations with customers, where the customer is either vulnerable (yes) or not vulnerable (no), we have provided the answer to each example for help.
    """

    example_string = ""
    for _, example_row in examples_df.iterrows():
        formatted_utterance = example_row['conversation_chunks']
        label_text = "Yes" if example_row['labels'] == 1 else "No"
        example_string += "\n" + "Example Conversation: " + formatted_utterance + "\nAnswer: " + label_text + "\n"



    question = "\n" + "Conversation to Classify: " + utterance + "\n"
    instruction = "\n Using the provided examples above for help, is the customer from the above conversation vulnerable based on our definiton of vulnerability, only answer Yes or No, and you must say only Yes or No."
    
    prompt = metaPrompt + example_string + question + instruction

    return prompt

In [6]:
def ask_gpt(question):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": question}
            ]
        )

        answer = response['choices'][0]['message']['content']
        tokens_used = response['usage']['total_tokens']
        input_tokens = len(question.split()) 
        return answer, tokens_used, input_tokens

    except openai.error.OpenAIError as e:
        return f"Error: {str(e)}", 0, 0
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}", 0, 0

In [8]:
def query_api(sampled_df, num_examples):
    results = []
    total_input_tokens = 0
    total_output_tokens = 0

    for _, row in sampled_df.iterrows():
        
        
        utterance = row['conversation_chunks']

        examples_df = df.sample(n=num_examples)
        prompt = get_template(utterance, examples_df)
       
        answer, tokens_used, input_tokens = ask_gpt(prompt)
        total_input_tokens += input_tokens
        total_output_tokens += tokens_used - input_tokens

        
        answer = answer.rstrip('.').strip()

        # convert the answer to a binary value 
        answer_binary = 1 if answer.lower() == "yes" else 0
        
        results.append({
            'Data': utterance,
            'Actual Label': row['labels'],
            'ChatGPT Label': answer_binary
        })
    
    cost_per_1k_input_tokens = 0.01
    cost_per_1k_output_tokens = 0.03
    total_cost = (total_input_tokens / 1000 * cost_per_1k_input_tokens) + \
                 (total_output_tokens / 1000 * cost_per_1k_output_tokens)

    print(f"Total Cost: {total_cost}")
    
    results_df = pd.DataFrame(results)
    
    return results_df


In [ ]:
#Testing
df_test = pd.read_csv("/Users/stevenslater/Desktop/FinalProject/Data/testwindowsimproved.csv")
df_test.head()

In [ ]:
results_test_12_examples = query_api(df_test, 12)

In [19]:
#save to csv
results_test_12_examples.to_csv("/Users/stevenslater/Desktop/FinalProject/Experiments/Experiement 4 - FSL/results_test_12_examples.csv")

In [ ]:
#METRICS 
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

actual_labels = results_test_8_examples['Actual Label'].astype(int) 
predicted_labels = results_test_8_examples['ChatGPT Label'].astype(int) 

accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

confusion = confusion_matrix(actual_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)
